<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/debugg/debugging_apis.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Se entrega a mas tardar el martes 14 de Noviembre a las 11:55 pm. Vale 40 puntos de tarea.
**Te recomiendo leer todo el ejercicio antes de empezar a hacer codigo, para que te des una idea de que formato y estructura necesitamos**

In [27]:
import pandas as pd
# Completa las librerias que necesites
import requests
from collections import Counter
import json

Objetivos del Ejercicio:

    Realizar una solicitud a la API de Pokémon para obtener datos sobre un Pokémon específico.
    Cargar y visualizar los datos en bruto para comprender su estructura.
    Limpiar y organizar los datos para un análisis posterior.
    Guardar los datos limpios en un archivo CSV.

El objetivo es crear un `DataFrame` (tidydata) de varios pokemones que ademas contengan metadata de sus moviemientos.  
Para ello deberas realizar request al `API` de pokemon, entender como vienen los datos y despues combinarlos con los `joins` correspondientes.

# API request

Crea codigo para acceder al API de pokemon y hacer requests. esta primera parte del codigo solo es para entender como es la estructura de los datos

Compelta el siguiente snippet para obter los datos del API de pokemon. La idea de estos primeros snippets es solo entender como funcionan los datos y como llegan. Para ello solo analizamos un pokemon y sus movimientos, mas adelante tenemos que repetirlo pero para varios pokemons.

## Pokemon Data

Para  el tipo de pokemon debes obtener:
1. `pokemon_id`
2. `pokemon_name`
3. `type`(s) cada pokemon tiene a 1 o dos tipos (no mas). Tienes que guardar ambos  

Guarda los `moves` del pokemon pues los usaremos mas adelante. En especial el `name` y la `url` de cada movimiento para que podamos iterar despues

In [1]:
# Elige tu pokemon
pokemon_name = 'lucario'

In [5]:
# Solicitud para obtener datos de un Pokémon
pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{pokemon_name}'
# Realizar la solicitud al API
pokemon_response = requests.get(pokemon_url)

# Verificar que la respuesta es exitosa
if pokemon_response.status_code == 200:
    # Convertir la respuesta en formato JSON
    pokemon_data = pokemon_response.json()

    # Obtener el ID y el nombre del Pokémon
    pokemon_id = pokemon_data['id']
    pokemon_name = pokemon_data['name']

    # Obtener los tipos del Pokémon
    types = [t['type']['name'] for t in pokemon_data['types']]

    # Obtener los movimientos del Pokémon
    moves = [{'name': move['move']['name'], 'url': move['move']['url']} for move in pokemon_data['moves']]

    # Crear un DataFrame para visualizar los datos
    df_pokemon = pd.DataFrame({
        'pokemon_id': [pokemon_id],
        'pokemon_name': [pokemon_name],
        'type1': [types[0]] if len(types) > 0 else [None],
        'type2': [types[1]] if len(types) > 1 else [None]
    })

    df_moves = pd.DataFrame(moves)

    print(df_pokemon)
    print(df_moves)
else:
    print("Error en la solicitud al API")
    
    # Lista para almacenar los metadatos de los movimientos
moves_metadata = []

# Iterar sobre cada movimiento
for move in moves:
    move_url = move['url']
    move_response = requests.get(move_url)

    if move_response.status_code == 200:
        move_data = move_response.json()

        # Aquí puedes elegir qué metadatos deseas extraer. Ejemplo: power, pp, type
        move_metadata = {
            'move_name': move['name'],
            'power': move_data.get('power', None),
            'pp': move_data.get('pp', None),
            'move_type': move_data['type']['name']
        }

        moves_metadata.append(move_metadata)
    else:
        print(f"Error al obtener datos del movimiento: {move['name']}")

# Convertir la lista de metadatos en un DataFrame
df_moves_metadata = pd.DataFrame(moves_metadata)
print(df_moves_metadata)


   pokemon_id pokemon_name     type1  type2
0         448      lucario  fighting  steel
              name                                  url
0       mega-punch    https://pokeapi.co/api/v2/move/5/
1        ice-punch    https://pokeapi.co/api/v2/move/8/
2    thunder-punch    https://pokeapi.co/api/v2/move/9/
3     swords-dance   https://pokeapi.co/api/v2/move/14/
4        mega-kick   https://pokeapi.co/api/v2/move/25/
..             ...                                  ...
105     steel-beam  https://pokeapi.co/api/v2/move/796/
106  terrain-pulse  https://pokeapi.co/api/v2/move/805/
107       coaching  https://pokeapi.co/api/v2/move/811/
108     tera-blast  https://pokeapi.co/api/v2/move/851/
109     trailblaze  https://pokeapi.co/api/v2/move/885/

[110 rows x 2 columns]
         move_name  power  pp move_type
0       mega-punch   80.0  20    normal
1        ice-punch   75.0  15       ice
2    thunder-punch   75.0  15  electric
3     swords-dance    NaN  20    normal
4        mega-ki

## Moves Data

Ahora para cada movimiento `move` del pokemon debes obtener los siguientes datos usando la `url`
1. `name`
2. `accuracy`
3. `type`
4. `power`
5. `pp`

In [14]:
# Lista para almacenar los detalles de cada movimiento
detailed_moves = []

for move in moves:  # Asegúrate de que 'moves' contiene los movimientos que has recolectado anteriormente
    move_url = move['url']
    moves_response = requests.get(move_url)

    if moves_response.status_code == 200:
        moves_data = moves_response.json()

        # Extracción de los datos requeridos
        move_details = {
            'name': moves_data['name'],
            'accuracy': moves_data.get('accuracy'),  # Algunos movimientos pueden no tener precisión definida
            'type': moves_data['type']['name'],
            'power': moves_data.get('power'),  # Algunos movimientos pueden no tener poder definido
            'pp': moves_data['pp']
        }

        detailed_moves.append(move_details)
    else:
        print(f"Error al obtener datos del movimiento: {move['name']}")

# Convertir la lista de detalles de movimientos en un DataFrame
df_detailed_moves = pd.DataFrame(detailed_moves)
print(df_detailed_moves)

# Asegúrate de que esta función se ejecuta correctamente
def get_move_details(move_url):
    response = requests.get(move_url)
    if response.status_code == 200:
        move_data = response.json()
        return {
            'name': move_data['name'],
            'accuracy': move_data.get('accuracy'),
            'type': move_data['type']['name'],
            'power': move_data.get('power'),
            'pp': move_data['pp']
        }
    else:
        print(f"Error al obtener datos del movimiento: {move_url}")
        return None

# Iterar sobre cada Pokémon y sus movimientos para obtener detalles
for pokemon, details in pokemon_data.items():
    move_details_list = []
    for move in details['moves']:
        move_details = get_move_details(move['url'])
        if move_details:
            move_details_list.append(move_details)
    # Asegúrate de que esta línea se ejecute correctamente y que la clave sea 'move_details'
    pokemon_data[pokemon]['move_details'] = move_details_list



              name  accuracy      type  power  pp
0       mega-punch      85.0    normal   80.0  20
1        ice-punch     100.0       ice   75.0  15
2    thunder-punch     100.0  electric   75.0  15
3     swords-dance       NaN    normal    NaN  20
4        mega-kick      75.0    normal  120.0   5
..             ...       ...       ...    ...  ..
105     steel-beam      95.0     steel  140.0   5
106  terrain-pulse     100.0    normal   50.0  10
107       coaching       NaN  fighting    NaN  10
108     tera-blast     100.0    normal   80.0  10
109     trailblaze     100.0     grass   50.0  20

[110 rows x 5 columns]


# API Request para varios pokemons

Ahora que sabes como funciona el `API` realiza las peticiones de los siguientes pokemones respecto a sus caracteristicas
y moviemientos, y guardalas en una lista, diciconario o tabla (lo que mas te acomode). 

In [8]:
pokemons =['lucario', 'pikachu', 'gengar', 'charizard', 'blastoise', 'venusaur', 'dragonite', 'mew', 'mewtwo', 'garchomp']

In [11]:
# Diccionario para almacenar los datos de cada Pokémon
pokemon_data = {}

for pokemon in pokemons:
    # Datos del Pokémon
    pokemon_url = f'https://pokeapi.co/api/v2/pokemon/{pokemon}'
    response = requests.get(pokemon_url)

    if response.status_code == 200:
        data = response.json()
        
        # Extracción de la información básica del Pokémon
        pokemon_details = {
            'id': data['id'],
            'name': data['name'],
            'types': [t['type']['name'] for t in data['types']],
            'moves': [{'name': move['move']['name'], 'url': move['move']['url']} for move in data['moves']]
        }

        # Agregar los detalles al diccionario
        pokemon_data[pokemon] = pokemon_details
    else:
        print(f"Error al obtener datos de {pokemon}")

# Aquí, pokemon_data contiene los datos de cada Pokémon y sus movimientos


# Evolucion a  Tabular

Ahora para cada request (`pokemon` y `movimiento`) convierte su informacion a datos tabulares. Toma en cuenta que vamos a realizar operaciones sobre esos DataFrames, por lo tanto tienes que elegir un formato adecuado.  
Para cada pokemon, y para cada movimiento debe existir un `DataFrame` o puedes guardarlos en uno solo, aunque es recomendable dejarlos por separado.

In [16]:
# Tu codigo aqui

# DataFrame para almacenar los detalles de los Pokémon
pokemon_columns = ['pokemon_id', 'pokemon_name', 'type1', 'type2']
pokemon_rows = []

# DataFrame para almacenar los detalles de los movimientos
moves_columns = ['pokemon_name', 'move_name', 'accuracy', 'type', 'power', 'pp']
moves_rows = []

for pokemon, details in pokemon_data.items():
    # Añadir detalles del Pokémon
    types = details['types']
    pokemon_rows.append([
        details['id'],
        pokemon,
        types[0] if len(types) > 0 else None,
        types[1] if len(types) > 1 else None
    ])
    
    # Añadir detalles de los movimientos
    for move in details['move_details']:
        moves_rows.append([
            pokemon,
            move['name'],
            move['accuracy'],
            move['type'],
            move['power'],
            move['pp']
        ])

# Crear los DataFrames
df_pokemon = pd.DataFrame(pokemon_rows, columns=pokemon_columns)
df_moves = pd.DataFrame(moves_rows, columns=moves_columns)

# Ahora tienes df_pokemon con los detalles de cada Pokémon y df_moves con los detalles de cada movimiento


# Union de Informacion

1. Obten el nombre y caracteristicas de todos lo movimientos que aparecen en por lo menos 3 pokemons de los de la lista (osea por lo menos 3 pokemons pueden realizar el movimiento), y guarda los datos de movimiento en un solo `DataFrame`

In [18]:
# Tu codigo aqui
# Contar la frecuencia de cada movimiento entre todos los Pokémon
movimientos_frecuentes = Counter([move['name'] for pokemon in pokemon_data.values() for move in pokemon['move_details']])

# Filtrar los movimientos que aparecen en al menos 3 Pokémon
movimientos_comunes = [mov for mov, count in movimientos_frecuentes.items() if count >= 3]

# DataFrame para almacenar los detalles de los movimientos comunes
movimientos_comunes_detalles = []

for pokemon in pokemon_data.values():
    for move in pokemon['move_details']:
        if move['name'] in movimientos_comunes:
            movimientos_comunes_detalles.append(move)

# Eliminar duplicados
movimientos_comunes_unicos = {v['name']:v for v in movimientos_comunes_detalles}.values()

# Crear el DataFrame
df_movimientos_comunes = pd.DataFrame(movimientos_comunes_unicos)

# Visualizar el DataFrame
print(df_movimientos_comunes)


                 name  accuracy      type  power  pp
0          mega-punch      85.0    normal   80.0  20
1           ice-punch     100.0       ice   75.0  15
2       thunder-punch     100.0  electric   75.0  15
3        swords-dance       NaN    normal    NaN  20
4           mega-kick      75.0    normal  120.0   5
..                ...       ...       ...    ...  ..
204  stomping-tantrum     100.0    ground   75.0  10
205      draco-meteor      90.0    dragon  130.0   5
206         iron-head     100.0     steel   80.0  15
207    chilling-water     100.0     water   50.0  20
208         power-gem     100.0      rock   80.0  20

[209 rows x 5 columns]


1.1 Salva los datos en un archivo csv

In [22]:
df_movimientos_comunes.to_csv('pokemon_moves_3.csv', index=False)


2. Obten el nombre y caracteristicas todos los moviemientos que no se repiten en ningun otro pokemon, osea son movimientos unicos para un solo pokemon

In [23]:
# Tu codigo aqui
# 2. Obtener los movimientos únicos

# Contar la frecuencia de cada movimiento entre todos los Pokémon
movimientos_frecuencia = Counter([move['name'] for pokemon in pokemon_data.values() for move in pokemon['move_details']])

# Filtrar los movimientos únicos
movimientos_unicos = [mov for mov, count in movimientos_frecuencia.items() if count == 1]

# DataFrame para almacenar los detalles de los movimientos únicos
movimientos_unicos_detalles = []

for pokemon in pokemon_data.values():
    for move in pokemon['move_details']:
        if move['name'] in movimientos_unicos:
            # Agregar el nombre del Pokémon al detalle del movimiento para identificar a qué Pokémon pertenece
            move['pokemon_name'] = pokemon['name']
            movimientos_unicos_detalles.append(move)

# Crear el DataFrame para movimientos únicos
df_movimientos_unicos = pd.DataFrame(movimientos_unicos_detalles)


2.2 Salvalos a un csv

In [24]:
df_movimientos_unicos.to_csv('unrepeated_moves.csv', index=False)


# Manejo de tipos

Para cada tipo de pokemon (fire, water, grass, etc) que aparecen en los datos. Obten una serie de dataframes que contengan a los pokemones que son de cada tipo. Y colocalos en un diccionarion donde la llave (`key`) es el tipo

In [29]:

# Extraer todos los tipos de Pokémon
tipos = set([t for pokemon in pokemon_data.values() for t in pokemon['types']])

# Crear un diccionario de DataFrames, uno por cada tipo
df_tipos = {}

for tipo in tipos:
    # Filtrar los Pokémon que pertenecen a este tipo
    pokemons_del_tipo = [pokemon for pokemon in pokemon_data.values() if tipo in pokemon['types']]
    
    # Crear un DataFrame para cada tipo
    df_tipos[tipo] = pd.DataFrame(pokemons_del_tipo)


Despues guarda ese diccionario en un json llamado `tipos.json`

In [30]:
# Tu codigo aqui
# Nonbre del json `pokemon_tipes.json`
# Guardar el diccionario en un archivo JSON
# Dado que los DataFrames no son serializables directamente a JSON, primero los convertiremos a un formato adecuado
json_data = {tipo: df.to_json() for tipo, df in df_tipos.items()}

with open('pokemon_tipes.json', 'w') as file:
    json.dump(json_data, file)


# Seleccion de Pokemones

Ahora crea un DataFrame que para cada pokemon unicamente contenga los movimientos que son de su tipo (osea si es tipo fuego y voldaor solo usa movimientos de tipo volador y fuego). Y crea una tabla donde tengan los siguientes datos en formato `tidydata`:
1. `pokemon_id`
2. `pokemon_name`
3. `type`(s) cada pokemon tiene a 1 o dos tipos (no mas). Tienes que guardar ambos  
4. `name` (del movimiento)
5. `accuracy`
6. `type`
7. `power`
8. `pp`

In [32]:
# Tu codigo aqui
# Lista para almacenar los datos filtrados
datos_filtrados = []

for pokemon, details in pokemon_data.items():
    # Obtener los tipos del Pokémon
    tipos_pokemon = details['types']
    
    for move in details['move_details']:
        # Incluir solo los movimientos que coinciden con el tipo del Pokémon
        if move['type'] in tipos_pokemon:
            datos_filtrados.append({
                'pokemon_id': details['id'],
                'pokemon_name': pokemon,
                'type1': tipos_pokemon[0] if len(tipos_pokemon) > 0 else None,
                'type2': tipos_pokemon[1] if len(tipos_pokemon) > 1 else None,
                'move_name': move['name'],
                'accuracy': move['accuracy'],
                'move_type': move['type'],
                'power': move['power'],
                'pp': move['pp']
            })

# Crear un DataFrame con los datos filtrados
df_movimientos_tipos = pd.DataFrame(datos_filtrados)

Finalmente guarda los datos en `.parquet`

In [33]:
# Nombre del archivo: tabla_pokemon.parquet
df_movimientos_tipos.to_parquet('tabla_pokemon.parquet', index=False)
